# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
# initialize the list of flow type we are capturing
flow_list = ["mortalidad animal", "robos", "matanza \/ consumo",
             "préstamos concedidos temporalmente \(animales\)",
             "venta de animales", "otros \(especificar\)", 
             "partos \(nacimientos\)", "préstamos recibidos", 
             "compra de animales", "otros \(especificar\)", "iii-3"]
translation_list = ["animal_death", "robbery", "slaughter_or_consumption",
                    "loans_given", "sold", "others_outflow", "births",
                    "loans_received", "purchased", "others_inflow"]
# column headers not including species column
column_headers = ["bovines", "sheep", "goats", "camelids",
                  "donkeys", "equines", "pigs"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_date_range(section_search):
    date_range = ""

    if len(section_search) == 1:
        section_string = section_search[0]

        date_regex = r"([\w ]+\d+ *[-–] *[\w ]+\d+)"
        date_search = re.findall(date_regex, section_string)

        if len(date_search) == 1:
            date_range = date_search[0]
        else:
            date_range = "UNENCOUNTERED FORMAT"
    else:
        date_range = "UNENCOUNTERED FORMAT"

    return date_range


def parse_section_3_2(section_search, flow_list,
                      translation_list, column_headers):
    section_3_2_data = []

    if len(section_search) == 1:
        section_string = section_search[0] + "iii-3"

        for i in range(len(flow_list)-1):
            item_dict = {}
            item_1 = flow_list[i]
            item_2 = flow_list[i+1]
            term = translation_list[i]

            row_search = find_between(item_1, item_2, section_string)

            item_dict["event"] = term

            if i <= 5:
                item_dict["flow_type"] = "outflow"
            else:
                item_dict["flow_type"] = "inflow"

            # if product row found
            if len(row_search) == 1:
                row_string = row_search[0]
                row_list = row_string.split("\n")
                row_list_len = len(row_list)

                if row_list_len >= 18:
                    # skip odd indexes (1, 3, 5, 7)
                    # drop first two rows (start count at -2)
                    count = -2
                    if (row_list[-1 * count] == ""
                       and row_list[-1 * count + 2] != ""):
                        count -= 2

                    for j in range(row_list_len):
                        if count > 12:
                            break
                        if count >= 0 and count % 2 == 0:
                            col = column_headers[count//2]
                            entry = row_list[j]
                            entry = entry.strip()
                            item_dict[col] = entry
                        count += 1

                else:
                    for col in column_headers:
                        item_dict[col] = "UNENCOUNTERED FORMAT"
            else:
                for col in column_headers:
                    item_dict[col] = "UNENCOUNTERED FORMAT"

            section_3_2_data.append(item_dict)
    else:
        section_3_2_data = [{}]

    return section_3_2_data

### Process the Whole File

In [ ]:
def process_files(source_directory, flow_list,
                  translation_list, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            data_dict = {}

            interviewee = parse_interviewee(contents)

            section_start = "iii-2"
            section_end = "iii-3"
            section_search = find_between(section_start, section_end, contents)

            date_range = parse_date_range(section_search)

            section_3_2_data = parse_section_3_2(section_search, flow_list,
                                                 translation_list, column_headers)

            if len(section_3_2_data) != 10:
                print(f"Unencountered Format: {process_file},"
                      + f"Length of data: {len(section_3_2_data)}")

            for row in section_3_2_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def transform_date_range(dataf):
    replace_dict = {"agosto 2018 - agosto 2019": "august 2018-2019",
                    "septiembre 2018 - septiembre 2019": "september 2018-2019",
                    "octubre 2018 – octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre 2019": "october 2018-2019"}
    dataf["date_range"] = dataf["date_range"].replace(replace_dict)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 7
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_columns(dataf):
    # standardize values
    map_dict = {"1 (parto)": "1",
                "2 (parto)": "2",
                "o": "0"}

    cols = dataf.columns
    for col in cols:
        dataf[col] = dataf[col].replace(map_dict)

    return dataf


def transform_values(dataf):
    dataf = transform_date_range(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, flow_list,
                       translation_list, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_columns)
            .pipe(handle_null_data)
            .pipe(transform_values))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame
Create the `dropped_df` and collect the rows which contain no non-null values

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

null_df = dropped_df.loc[:, ["filename", "event"]]

### Check Dropped Row

In [ ]:
def get_dropped_row(section_search, event,
                    flow_list, translation_list):
    dropped_row = {}

    if len(section_search) == 1:
        section_string = section_search[0] + "iii-3"

        dropped_row["event"] = event

        flow_index = translation_list.index(event)
        item_1 = flow_list[flow_index]
        item_2 = flow_list[flow_index + 1]

        row_search = find_between(item_1, item_2, section_string)

        # if product row found
        if len(row_search) == 1:
            row_string = row_search[0]
            row_list = row_string.split("\n")
            row_list_len = len(row_list)

            if row_list_len >= 18:
                count = -2
                for j in range(row_list_len):
                    if count > 12:
                        break
                    if count >= 0:
                        col = str(count)
                        entry = row_list[j]
                        entry = entry.strip()
                        dropped_row[col] = entry
                    count += 1

            else:
                for j in range(12):
                    dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
        else:
            for j in range(12):
                dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
    else:
        dropped_row = {}

    return dropped_row

### Check Dropped Rows for Parsing Errors

In [ ]:
def check_dropped_rows(source_directory, flow_list, translation_list, s):
    process_file = s["filename"]
    event = s["event"]

    check_dict = {}

    file_path = os.path.join(source_directory, process_file)

    # with statements automatically control the closing of files
    with open(file_path, "r") as file:
        contents = file.read()
        contents = contents.lower()

        section_start = "iii-2"
        section_end = "iii-3"
        section_search = find_between(section_start, section_end, contents)

        dropped_row = get_dropped_row(section_search, event,
                                      flow_list, translation_list)

        # check to make sure files contain the right amount of entries
        if len(dropped_row) <= 1:
            print(f"Unencountered Format: {process_file}\n"
                  + f"Length of data: {len(dropped_row)}\n")

        check_dict["filename"] = process_file

        for key, value in dropped_row.items():
            check_dict[key] = value

    # convert list to Series
    check_series = pd.Series(data=check_dict)

    return check_series

In [ ]:
check_df = null_df.apply(lambda s: check_dropped_rows(source_directory,
                                                      flow_list,
                                                      translation_list,
                                                      s),
                         axis=1)

In [ ]:
def drop_null_check(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 13
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf

In [ ]:
non_null_df = (check_df
               .pipe(start_pipeline)
               .pipe(drop_null_check))

In [ ]:
non_null_df

## Load
### Export to CSV

In [ ]:
if non_null_df.shape[0] == 0:
    datasets_directory = "../datasets/"
    filename = "section_3_2.csv"
    file_path = os.path.join(datasets_directory, filename)

    clean_df.to_csv(file_path, index=False, na_rep="null")

    print(f"Exported to {filename}")
else:
    print("Error: Dropped rows contain non-null values")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]